In [1]:
import pandas as pd
from pymongo import MongoClient
import pymongo
import numpy as np

In [2]:
class ConectColl:
    def __init__(self):
        self.client = MongoClient()
        self.db = self.client['Bycicle_Accidents']
        self.acc = self.db['Historical']
        self.test = self.db['Test']

coll = ConectColl()
bici = pd.read_csv("bici_clean.csv")

In [3]:
# bici_test = bici.loc[(bici.horario=='Tarde')&(bici.festividad=='Laborable')&(bici.meteo=='Lluvia')]

In [4]:
# bici_test.head()

In [5]:
# min_lat=int((bici_test.lat.min()-0.01)*400)
# max_lat=int((bici_test.lat.max()+0.01)*400)

# min_lon=int((bici_test.lon.min()-0.01)*300)
# max_lon=int((bici_test.lon.max()+0.01)*300)


In [6]:
# no_damage = pd.DataFrame(columns=['horario','festividad','lesividad','meteo','lon', 'lat'])

# for lat in range(min_lat, max_lat):
#     for lon in range (min_lon, max_lon):
#         no_damage = no_damage.append({
#             'horario':'Tarde',
#             'festividad':'Laborable',
#             'lesividad':'No',
#             'meteo':'Lluvia',
#             'lon':lon/300,
#             'lat':lat/400,
#         }, ignore_index=True)



In [25]:
lst_h = list(bici.horario.unique())
lst_f = list(bici.festividad.unique())
lst_m = list(bici.meteo.unique())

no_damage = pd.DataFrame(columns=['horario','festividad','lesividad','meteo','lon', 'lat'])

for h in lst_h:
    for f in lst_f:
        for m in lst_m:
            bici_test = bici.loc[(bici.horario==h)&(bici.festividad==f)&(bici.meteo==m)]
            
            min_lat=int((bici_test.lat.min()-0.0002)*100)
            max_lat=int((bici_test.lat.max()+0.0002)*100)

            min_lon=int((bici_test.lon.min()-0.00015)*75)
            max_lon=int((bici_test.lon.max()+0.00015)*75)
            
            for lat in range(min_lat, max_lat):
                for lon in range (min_lon, max_lon):
                    no_damage = no_damage.append({
                        'horario':h,
                        'festividad':f,
                        'lesividad':'No',
                        'meteo':m,
                        'lon':lon/75,
                        'lat':lat/100,
                    }, ignore_index=True)

In [26]:
no_damage.shape

(2403, 6)

In [17]:
no_damage.shape

(9825, 7)

In [10]:
bici.head()

,fecha,dia,year,horario,festividad,tipo_accidente,lesividad,meteo,distrito,direccion,lon,lat
0,2019-01-01 14:00:00,2019-01-01 00:00:00,2019,Tarde,Festivo,Colision,Moderada,Despejado,SALAMANCA,CALLE CASTELLO / CALLE DON RAMON DE LA CRUZ Ma...,-3.681209,40.428990
1,2019-01-02 21:00:00,2019-01-02 00:00:00,2019,Noche,Laborable,Colision,Grave,Despejado,HORTALEZA,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALE...,-3.654527,40.466921
2,2019-01-03 14:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,CALLE FELIPE ALVAREZ 10 Madrid,-3.623423,40.381971
3,2019-01-03 13:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,-3.608276,40.377939
4,2019-01-03 13:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,-3.608276,40.377939


In [27]:
num_lon = 1/75
num_lat = 1/100

no_damage['num_acc']=int()
lst_ =list()
for i, r in no_damage.iterrows():
    accs = len(bici.loc[
        (bici.horario==r['horario'])&(bici.festividad==r['festividad'])&(bici.meteo==r['meteo'])&
        (bici.lon>=(r['lon']-num_lon))&(bici.lon<=(r['lon']+num_lon))&
        (bici.lat>=(r['lat']-num_lat))&(bici.lat<=(r['lat']+num_lat))])
    no_damage.loc[i,'num_acc']=accs
    

In [28]:
no_damage.num_acc.value_counts().head()
#75/100

0    942
1    479
2    276
3    163
4    109
Name: num_acc, dtype: int64

In [22]:
no_damage.num_acc.value_counts().head()
#200/150

0    6580
1    1459
2     611
3     295
4     175
Name: num_acc, dtype: int64

In [32]:
bici.loc[(bici.horario=='Tarde')&(bici.festividad=='Laborable')&(bici.meteo=='Despejado')&
        (bici.lon>(-3.60-0.01))&(bici.lon<(-3.60+0.01))&
        (bici.lat>(40.3-0.1))&(bici.lat<(40.3+0.1))]

,fecha,dia,year,horario,festividad,tipo_accidente,lesividad,meteo,distrito,direccion,lon,lat
3,2019-01-03 13:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,-3.608276,40.377939
4,2019-01-03 13:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,-3.608276,40.377939
1271,2018-09-25 14:00:00,2018-09-25 00:00:00,2018,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,CALLE DE PUENTELARRA NUM ...,-3.607905,40.382831
1760,2017-06-08 14:00:00,2017-06-08 00:00:00,2017,Tarde,Laborable,Colision,Grave,Despejado,VICALVARO,CALLE DE LA PIROTECNIA - CALLE DE LOS SONDEOS ...,-3.590250,40.395814
1816,2017-06-28 18:00:00,2017-06-28 00:00:00,2017,Tarde,Laborable,Colision,Leve,Despejado,VICALVARO,CALLE DE LA PIROTECNIA - CALLE DE LOS SONDEOS ...,-3.590250,40.395814
1918,2017-08-21 19:00:00,2017-08-21 00:00:00,2017,Tarde,Laborable,Imprudencia,Leve,Despejado,VILLA DE VALLECAS,CALLE REAL DE ARGANDA - CALLE DE POZA DE LA S...,-3.606802,40.378183
1931,2017-08-30 20:00:00,2017-08-30 00:00:00,2017,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,CALLE REAL DE ARGANDA NUM ...,-3.604064,40.376095
2063,2017-10-19 19:00:00,2017-10-19 00:00:00,2017,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,AVENIDA DE SANTA EUGENIA - CALLE REAL DE ARGA...,-3.608276,40.377939


In [26]:
-7<-4


True

In [27]:
1/300*3/4/2

0.00125

In [8]:
def geopoint(lon, lat):
    return {"type": "Point", "coordinates": [lon, lat]}


In [9]:
no_damage['Localizacion'] = np.vectorize(geopoint)(no_damage['lon'], no_damage['lat'])

coll.test.insert_many(no_damage.to_dict('record'))
coll.test.create_index([('Localizacion', pymongo.GEOSPHERE)])

'Localizacion_2dsphere'

In [ ]:
def num_horario(value):
    dict_hora = {
        'Mañana': 1,
        'Tarde': 2,
        'Noche': 3
    }
    return dict_hora[value]

def num_fest(value):
    dict_fest = {
        'Laborable': 1,
        'Vispera': 2,
        'Festivo': 3
    }
    return dict_fest[value]

def prepare_df(data):
    data.drop(['fecha', 'dia', 'year', 'tipo_accidente',
               'distrito', 'direccion'], axis=1, inplace=True)
    data.horario = data.horario.apply(lambda x: num_horario(x))
    data.festividad = data.festividad.apply(lambda x: num_fest(x))
    data = pd.get_dummies(data, columns=['meteo'], prefix_sep='', prefix='')
    return data

In [5]:
data = prepare_df(bici)

In [5]:
bici.head()

,fecha,dia,year,horario,festividad,tipo_accidente,lesividad,meteo,distrito,direccion,lon,lat
0,2019-01-01 14:00:00,2019-01-01 00:00:00,2019,Tarde,Festivo,Colision,Moderada,Despejado,SALAMANCA,CALLE CASTELLO / CALLE DON RAMON DE LA CRUZ Ma...,-3.681209,40.428990
1,2019-01-02 21:00:00,2019-01-02 00:00:00,2019,Noche,Laborable,Colision,Grave,Despejado,HORTALEZA,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALE...,-3.654527,40.466921
2,2019-01-03 14:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,CALLE FELIPE ALVAREZ 10 Madrid,-3.623423,40.381971
3,2019-01-03 13:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,-3.608276,40.377939
4,2019-01-03 13:00:00,2019-01-03 00:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,-3.608276,40.377939


In [5]:
all_acc = coll.acc.find()
df = pd.DataFrame(all_acc)

In [6]:
df.head()

,_id,fecha,year,horario,festividad,tipo_accidente,lesividad,meteo,distrito,direccion,localizacion
0,5e11db4d64114c18c0e3a5c2,2019-01-01 14:00:00,2019,Tarde,Festivo,Colision,Moderada,Despejado,SALAMANCA,CALLE CASTELLO / CALLE DON RAMON DE LA CRUZ Ma...,"{'type': 'Point', 'coordinates': [-3.681209, 4..."
1,5e11db4d64114c18c0e3a5c3,2019-01-02 21:00:00,2019,Noche,Laborable,Colision,Grave,Despejado,HORTALEZA,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALE...,"{'type': 'Point', 'coordinates': [-3.654527, 4..."
2,5e11db4d64114c18c0e3a5c4,2019-01-03 14:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,CALLE FELIPE ALVAREZ 10 Madrid,"{'type': 'Point', 'coordinates': [-3.623423, 4..."
3,5e11db4d64114c18c0e3a5c5,2019-01-03 13:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,"{'type': 'Point', 'coordinates': [-3.608276, 4..."
4,5e11db4d64114c18c0e3a5c6,2019-01-03 13:00:00,2019,Tarde,Laborable,Colision,Leve,Despejado,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,"{'type': 'Point', 'coordinates': [-3.608276, 4..."


In [7]:
lluvia_tarde = coll.acc.find({'$and':[{"horario": "Tarde"}, {'meteo':'Lluvia'}]})
df_llt = pd.DataFrame(lluvia_tarde)

In [8]:
df_llt.head()

,_id,fecha,year,horario,festividad,tipo_accidente,lesividad,meteo,distrito,direccion,localizacion
0,5e11db4d64114c18c0e3a5ee,2019-01-29 16:00:00,2019,Tarde,Laborable,Caida,Moderada,Lluvia,CENTRO,PASEO. RECOLETOS 23 Madrid,"{'type': 'Point', 'coordinates': [-3.692396, 4..."
1,5e11db4e64114c18c0e3a678,2019-04-02 14:00:00,2019,Tarde,Laborable,Colision,Leve,Lluvia,FUENCARRAL-EL PARDO,"AUTOV. M-607, +01500S Madrid","{'type': 'Point', 'coordinates': [-3.691782, 4..."
2,5e11db4e64114c18c0e3a679,2019-04-02 14:00:00,2019,Tarde,Laborable,Colision,Leve,Lluvia,FUENCARRAL-EL PARDO,"AUTOV. M-607, +01500S Madrid","{'type': 'Point', 'coordinates': [-3.691782, 4..."
3,5e11db4e64114c18c0e3a67f,2019-04-06 20:00:00,2019,Tarde,Vispera,Caida,Moderada,Lluvia,CENTRO,CALLE VICTORIA 4 Madrid,"{'type': 'Point', 'coordinates': [-3.701736, 4..."
4,5e11db4e64114c18c0e3a686,2019-04-09 13:00:00,2019,Tarde,Laborable,Otro,Leve,Lluvia,CENTRO,PASEO. RECOLETOS / CALLE GENOVA Madrid,"{'type': 'Point', 'coordinates': [-3.690993, 4..."


In [ ]:
.find({'$and': [query, {"location": {"$near": {"$geometry": lst_loc, "$maxDistance": radio}}}]}))